In [1]:
import os
import sys
root = '/home/mark/Documents/code/mdrp'
sys.path.append(root)

import pickle
import numpy as np
import pickle
from utils.nlp_mdrp import mdrp_prob
import cyipopt

from utils.convert import convert_instance
from utils.nlp_mdrp import mdrp_prob
from utils.helpers import *


In [2]:
def main(parameters):
    # Setup the directory and logger    
    # Load the instance

    isinstance_path = os.path.join(root+'/instances/',parameters['inst_path'])
    prob_setup = convert_instance(isinstance_path,parameters)
    # add additional parameters
    prob_setup['max_rho'] =  parameters['max_rho']
    prob_setup['demand']  =  parameters['demand']
    prob_setup['scale']   =  parameters['scale']
    # SOLVE NLP
    mdrp = mdrp_prob(prob_setup)
    lb, ub, cl, cu = mdrp.get_bounds()
    x0 = mdrp.guess_init()
    nlp = cyipopt.Problem(
        n=len(x0),
        m=len(cl),
        problem_obj=mdrp,
        lb=lb,
        ub=ub,
        cl=cl,
        cu=cu
        )

    # nlp.addOption('derivative_test', 'second-order')
    nlp.add_option('mu_strategy', 'adaptive')
    nlp.add_option('tol', 1e-4)

    x, info = nlp.solve(x0)

    # Eval
    return x, mdrp

In [3]:
parameters = {}
parameters['seed'] = 0
parameters['speeds'] = [320,804.67,107.29] 
parameters['u_ratios'] = [1,1,1] # identical service time
parameters['max_dph'] = 100
parameters['min_dph'] = 10

# parameters['demand'] = 0.99
# parameters['scale'] = True # if true then uses demand value as portion of max demand
parameters['scale'] = False
parameters['demand'] = 275

parameters['inst_path'] = '0o100t100s1p100'
parameters['cost'] = [20,10,5]

# costs are set arbitrarily large to not allow for cars
N_values = ([100,1,1],[20,24,100])

In [4]:
# # QUICKLY CHECK FOR MAX DEMAND
# # NOTE: This cell is only relevant when scale parameter is true

# N_values = ([100,1,1],[20,24,100])

# for N, i in zip(N_values, range(len(N_values))):
#     # Assumes first is only cars
#     if i == 0:
#         parameters['max_rho'] = [0.9,0,0]
#     else:
#         parameters['max_rho'] = [0.9,0.8,0.9]
#     parameters['N'] = N
#     # load the instance
#     isinstance_path = os.path.join(root+'/instances/',parameters['inst_path'])
#     prob_setup = convert_instance(isinstance_path,parameters)
#     # add additional parameters
#     prob_setup['max_rho'] =  parameters['max_rho']
#     prob_setup['demand']  =  parameters['demand']
#     prob_setup['scale']   =  parameters['scale']
#     # SOLVE NLP
#     mdrp = mdrp_prob(prob_setup)
#     print('----------------'*4)
#     print(N)
#     print(mdrp.tot_demand)
#     # print('wait time: ',prob_setup['q_time']*60)
#     # print('service time: ',60/prob_setup['mu'])


In [5]:
# RUN THE OPTIMIZATIONS

# results = []
# for N, i in zip(N_values, range(len(N_values))):
#     # Assumes first is only cars
#     if i == 0:
#         parameters['max_rho'] = [0.9,0,0]
#     else:
#         parameters['max_rho'] = [0.9,0.8,0.9]
#     result = {}
#     print('Running iter %d'%i)
#     parameters['N'] = N
#     sol, mdrp = main(parameters)
#     result['sol'] = sol
#     result['mdrp'] = mdrp
#     result['parameters'] = parameters
#     results.append(result)



In [6]:
# SAVE THE RESULTS

# save_dir = os.path.join(root, 'results', parameters['inst_path'])
# try: os.mkdir(save_dir)
# except: pass
# pickle.dump(results, open(save_dir+'/results_4.p','wb'))

In [7]:
# EVALUATE

results_path = '/home/mark/Documents/code/mdrp/results/0o100t100s1p100/results_4.p'
results = pickle.load(open(results_path,'rb'))
# delivery_rate = 1/(np.array([[18.99,np.inf,np.inf],[23.57,14.66,26.46]])/60)
# costs = [[20, 10, 5],[20, 10, 5],[20, 10, 5]]
for i in range(len(results)):
    # find prices
    result = results[i]
    mdrp = result['mdrp']
    # cost per order
    mdrp.cost = [5,5,1.5]
    # prices, min_prices = eval_sol(result)

    # check nash condition
    x       = result['sol']
    flow    = mdrp.vec_to_mat(np.round(x,4))
    lat     = mdrp.get_latency(flow)
    prices, min_prices = eval_sol(result)

    count = 0
    for i in range(mdrp.n):
        if not np.all(np.argsort(lat[i])[::-1] ==  np.argsort(prices[i])):
            # print('prices: ', prices[i])
            # print('lats: ',lat[i])
            count += 1
    
    print('total off: ',count)



Problem Evlatuation
--------------------------------------------------------
Orders Demanded (order/hour): 275.00
--------------------------------------------------------
Mode 		|car	|drone	|droid	|total	|
----------------|-------|-------|-------|-------|
N (# veh.)	|100	|1	|1	|
Orders (%)	|100.01	|0.00	|0.00	|100.01	|
Utilization (%)	|0.88	|0.00	|0.00	|0.87	|
D Time(min.)	|18.99	|nan	|nan	|18.99	|
Order Price	|5.00	|nan	|nan	|5.00	|
Op. Cost ($/hr)	|1375.12	|0.00	|0.00	|1375.12	|
Revenue ($/hr)	|1375.23	|0.00	|0.00	|1375.23	|
Distance	|1.47	|nan	|nan	|1.47	|
--------------------------------------------------------
Minimum Price ($): 1.85
--------------------------------------------------------
total off:  0


Problem Evlatuation
--------------------------------------------------------
Orders Demanded (order/hour): 275.00
--------------------------------------------------------
Mode 		|car	|drone	|droid	|total	|
----------------|-------|-------|-------|-------|
N (# veh.)	|20	|24	|10

/home/mark/Documents/code/mdrp/utils/helpers.py:108: RuntimeWarning: invalid value encountered in scalar divide
  lat_j = np.inner(lat[:,j],flow[:,j])/(orders_j[j]*tot_demand)
/home/mark/Documents/code/mdrp/utils/helpers.py:117: RuntimeWarning: invalid value encountered in scalar divide
  price_j = np.inner(prices[:,j],flow[:,j])/(orders_j[j]*tot_demand)
/home/mark/Documents/code/mdrp/utils/helpers.py:150: RuntimeWarning: invalid value encountered in scalar divide
  dist_j = np.inner(distance[:,j],flow[:,j])/(orders_j[j]*tot_demand*1000)
